# 🚀 Enhanced Breakout Scanner Dashboard - Unified Integration

Interactive visualization dashboard for breakout scanner performance analysis using the new unified DuckDB integration.
Shows scanner results with CLI-style execution and comprehensive performance metrics.

In [ ]:
# Setup and Imports - Updated for Unified Integration
import sys
import os
from pathlib import Path
from datetime import datetime, date, time, timedelta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import subprocess
import json

# Add project root to Python path
current_dir = Path.cwd()
if current_dir.name == 'scanner':
    project_root = current_dir.parent.parent
elif current_dir.name == 'notebook':
    project_root = current_dir.parent
else:
    project_root = current_dir

sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'src'))

# Import unified scanner integration
try:
    from src.app.startup import get_scanner
    from src.infrastructure.adapters.scanner_read_adapter import DuckDBScannerReadAdapter
    from src.infrastructure.database.unified_duckdb import UnifiedDuckDBManager, DuckDBConfig
    print("✅ Unified scanner integration imported successfully!")
except ImportError as e:
    print(f"⚠️  Unified imports failed: {e}")
    print("   Using CLI fallback approach...")

# Set up dark mode theme for plots
pio.templates.default = "plotly_dark"

# Custom dark theme configuration
dark_template = {
    "layout": {
        "paper_bgcolor": "#1e1e1e",
        "plot_bgcolor": "#1e1e1e",
        "font": {"color": "#ffffff"},
        "xaxis": {
            "gridcolor": "#333333",
            "linecolor": "#666666",
            "tickcolor": "#666666",
            "tickfont": {"color": "#ffffff"}
        },
        "yaxis": {
            "gridcolor": "#333333",
            "linecolor": "#666666",
            "tickcolor": "#666666",
            "tickfont": {"color": "#ffffff"}
        }
    }
}

# CLI-style helper functions
def run_cli_command(command, cwd=None):
    """Run CLI command and return output"""
    try:
        result = subprocess.run(command, shell=True, capture_output=True, text=True, cwd=cwd)
        return result.returncode, result.stdout, result.stderr
    except Exception as e:
        return -1, "", str(e)

def run_scanner_cli(scanner_type='breakout', date_str=None):
    """Run scanner using CLI approach"""
    date_param = f", date_str='{date_str}'" if date_str else ""
    cmd = f"""cd {project_root} && python -c "
import sys
sys.path.insert(0, 'src')
from src.app.startup import get_scanner
from datetime import date, time
import json

try:
    scanner = get_scanner('{scanner_type}')
    scan_date = date.today() if not '{date_str}' else date.fromisoformat('{date_str}')
    results = scanner.scan(scan_date=scan_date, cutoff_time=time(9, 50))
    
    if hasattr(results, 'to_dict') and len(results) > 0:
        results_dict = results.to_dict('records')
        print(json.dumps(results_dict))
    else:
        print('[]')
except Exception as e:
    print(f'[]')
    """"
    
    return_code, stdout, stderr = run_cli_command(cmd)
    if return_code == 0:
        try:
            return json.loads(stdout.strip())
        except:
            return []
    else:
        print(f"CLI Error: {stderr}")
        return []

print("✅ Dashboard setup complete with unified integration!")
print("🔧 CLI-style execution functions ready")

In [ ]:
# Initialize Scanner with Unified Integration
print("🔧 Initializing Breakout Scanner with Unified Integration...")

# Try unified approach first
try:
    scanner = get_scanner('breakout')
    print("✅ Unified scanner created successfully!")
    print(f"Scanner type: {type(scanner).__name__}")
    
    # Test the unified connection
    from src.infrastructure.database.unified_duckdb import UnifiedDuckDBManager, DuckDBConfig
    config = DuckDBConfig(database_path='financial_data.duckdb')
    manager = UnifiedDuckDBManager(config)
    print("✅ Unified DuckDB manager initialized")
    
    # Check connection pool stats
    pool_stats = manager.get_connection_stats()
    print(f"🔗 Connection pool: {pool_stats.get('active_connections', 0)}/{pool_stats.get('max_connections', 0)} connections")
    
except Exception as e:
    print(f"⚠️  Unified initialization failed: {e}")
    print("🔄 Falling back to CLI approach...")

print("✅ Scanner initialization complete!")

In [ ]:
# Run Scanner Analysis - CLI Style
print("🆕 Running Breakout Scanner Analysis...")

# Configure analysis parameters
start_date = date.today() - timedelta(days=7)
end_date = date.today()
scanner_type = 'breakout'

print(f"📅 Analyzing period: {start_date} to {end_date}")
print(f"🎯 Scanner type: {scanner_type}")
print(f"⏰ Breakout detection: 09:50:00")

# Run analysis for each day in the range
all_results = []
current_date = start_date

while current_date <= end_date:
    print(f"\n📊 Scanning {current_date}...")
    
    try:
        # Try unified approach first
        if 'scanner' in locals():
            day_results = scanner.scan(scan_date=current_date, cutoff_time=time(9, 50))
            if hasattr(day_results, 'to_dict') and len(day_results) > 0:
                day_results_list = day_results.to_dict('records')
            else:
                day_results_list = []
        else:
            # Fall back to CLI approach
            day_results_list = run_scanner_cli(scanner_type, current_date.isoformat())
        
        # Add date to results
        for result in day_results_list:
            result['scan_date'] = current_date.isoformat()
        
        all_results.extend(day_results_list)
        print(f"✅ Found {len(day_results_list)} breakout candidates")
        
    except Exception as e:
        print(f"⚠️  Error scanning {current_date}: {e}")
        print("   Continuing with next date...")
    
    current_date += timedelta(days=1)

# Convert to DataFrame for analysis
if all_results:
    df = pd.DataFrame(all_results)
    print(f"\n📈 Total results collected: {len(df)} breakout opportunities")
    print(f"📊 DataFrame shape: {df.shape}")
    print(f"📋 Available columns: {list(df.columns)}")
    
    # Basic statistics
    if len(df) > 0:
        print(f"\n📊 Basic Statistics:")
        print(f"  • Unique symbols: {df['symbol'].nunique() if 'symbol' in df.columns else 'N/A'}")
        print(f"  • Date range: {df['scan_date'].min()} to {df['scan_date'].max()}")
        
        if 'close' in df.columns:
            print(f"  • Average breakout price: ₹{df['close'].mean():.2f}")
        if 'volume' in df.columns:
            print(f"  • Average volume: {df['volume'].mean():,.0f}")
else:
    print("\n⚠️  No breakout results found")
    print("   This may be due to limited market data or strict criteria")
    df = pd.DataFrame()  # Empty DataFrame for compatibility

print("\n🎉 Breakout scanner analysis complete!")

# 📊 Performance Dashboard

Interactive 2x3 grid visualization of breakout scanner performance using unified integration.

In [ ]:
# Performance Dashboard with Plotly 2x3 Grid Layout
print("📊 Creating Performance Dashboard...")

# Check if we have results to analyze
if 'df' not in locals() or df.empty:
    print("❌ No results available. Please run the Scanner Analysis cell first.")
    print("💡 Make sure you have market data in your database.")
else:
    print(f"📈 Analyzing {len(df)} breakout opportunities...")
    
    # Create 2x3 subplot layout
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=(
            'Performance Summary', 'Daily Performance', 'Success Distribution',
            'Top Performers', 'Price Change Distribution', 'Volume vs Price Scatter'
        ),
        specs=[
            [{'type': 'table'}, {'type': 'bar'}, {'type': 'pie'}],
            [{'type': 'table'}, {'type': 'histogram'}, {'type': 'scatter'}]
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.05
    )
    
    # ===== 1. PERFORMANCE SUMMARY TABLE =====
    total_breakouts = len(df)
    
    summary_data = {
        'Metric': ['Total Breakouts', 'Unique Symbols', 'Date Range', 'Avg Breakout Price', 'Total Volume'],
        'Value': [
            total_breakouts,
            df['symbol'].nunique() if 'symbol' in df.columns else 'N/A',
            f"{df['scan_date'].min()} to {df['scan_date'].max()}" if 'scan_date' in df.columns else 'N/A',
            f"₹{df['close'].mean():.2f}" if 'close' in df.columns else 'N/A',
            f"{df['volume'].sum():,.0f}" if 'volume' in df.columns else 'N/A'
        ]
    }
    
    fig.add_trace(
        go.Table(
            header=dict(values=['<b>Performance Metrics</b>', '<b>Values</b>'],
                      fill_color='#2E4F7F', align='left', font=dict(size=12, color='white')),
            cells=dict(values=[summary_data['Metric'], summary_data['Value']],
                     fill_color='#1e1e1e', align='left', font=dict(size=11, color='white'))
        ),
        row=1, col=1
    )
    
    # ===== 2. DAILY PERFORMANCE BAR CHART =====
    if 'scan_date' in df.columns:
        try:
            daily_stats = df.groupby('scan_date').size().reset_index(name='count')
            
            fig.add_trace(
                go.Bar(
                    x=daily_stats['scan_date'],
                    y=daily_stats['count'],
                    marker_color='green',
                    text=daily_stats['count'],
                    textposition='auto',
                    name='Breakouts Found'
                ),
                row=1, col=2
            )
            
        except Exception as e:
            print(f"Note: Could not create daily chart: {e}")
    
    # ===== 3. SYMBOL DISTRIBUTION PIE CHART =====
    if 'symbol' in df.columns:
        symbol_counts = df['symbol'].value_counts().head(10)  # Top 10 symbols
        
        fig.add_trace(
            go.Pie(
                labels=symbol_counts.index,
                values=symbol_counts.values,
                textinfo='label+percent',
                title='Top 10 Symbols'
            ),
            row=1, col=3
        )
    
    # ===== 4. TOP PERFORMERS TABLE =====
    if 'symbol' in df.columns and 'volume' in df.columns:
        top_performers = df.nlargest(5, 'volume')[['symbol', 'volume', 'close']]
        table_data = []
        
        for _, row in top_performers.iterrows():
            symbol = row['symbol']
            volume = f"{row['volume']:,.0f}"
            price = f"₹{row['close']:.2f}"
            table_data.append([symbol, volume, price])
        
        fig.add_trace(
            go.Table(
                header=dict(values=['<b>Symbol</b>', '<b>Volume</b>', '<b>Price</b>'],
                          fill_color='#2E8B57', align='left', font=dict(size=11, color='white')),
                cells=dict(values=list(zip(*table_data)) if table_data else [['N/A'], ['N/A'], ['N/A']],
                         fill_color='#1e1e1e', align='left', font=dict(size=10, color='white'))
            ),
            row=2, col=1
        )
    
    # ===== 5. PRICE DISTRIBUTION =====
    if 'close' in df.columns:
        fig.add_trace(
            go.Histogram(
                x=df['close'],
                nbinsx=20,
                marker_color='blue',
                opacity=0.7,
                name='Breakout Prices'
            ),
            row=2, col=2
        )
    
    # ===== 6. VOLUME VS PRICE SCATTER =====
    if 'volume' in df.columns and 'close' in df.columns:
        fig.add_trace(
            go.Scatter(
                x=df['volume'],
                y=df['close'],
                mode='markers',
                marker=dict(
                    size=8,
                    color='orange',
                    showscale=False
                ),
                name='Volume vs Price',
                text=df['symbol']
            ),
            row=2, col=3
        )
    
    # Update layout
    fig.update_layout(
        height=800,
        width=1200,
        title_text='<b>🚀 Breakout Scanner Performance Dashboard - Unified Integration</b>',
        title_x=0.5,
        title_font=dict(size=20, color='white'),
        showlegend=True,
        template='plotly_dark',
        paper_bgcolor='#1e1e1e',
        plot_bgcolor='#1e1e1e'
    )
    
    # Update subplot titles and axes with dark mode styling
    fig.update_xaxes(title_text='Date', row=1, col=2, title_font=dict(color='white'))
    fig.update_yaxes(title_text='Count', row=1, col=2, title_font=dict(color='white'))
    fig.update_xaxes(title_text='Breakout Price (₹)', row=2, col=2, title_font=dict(color='white'))
    fig.update_yaxes(title_text='Frequency', row=2, col=2, title_font=dict(color='white'))
    fig.update_xaxes(title_text='Volume', row=2, col=3, title_font=dict(color='white'))
    fig.update_yaxes(title_text='Price (₹)', row=2, col=3, title_font=dict(color='white'))
    
    # Apply dark mode to all axes
    fig.update_xaxes(gridcolor='#333333', linecolor='#666666', tickfont=dict(color='white'))
    fig.update_yaxes(gridcolor='#333333', linecolor='#666666', tickfont=dict(color='white'))
    
    # Show the dashboard
    fig.show()
    
    print("\n✅ Performance dashboard created successfully!")
    print(f"📊 Analyzed {total_breakouts} breakout opportunities")
    print(f"🎯 Using unified DuckDB integration")

print("\n🎉 Performance Dashboard Complete!")

# 📈 CLI Integration Demo

Demonstrate running scanners via command line interface.

In [ ]:
# CLI Integration Demo
print("🖥️  CLI Integration Demo")
print("=" * 50)

# Test CLI scanner execution
test_date = date.today().isoformat()

print(f"Running breakout scanner for {test_date}...")
cli_results = run_scanner_cli('breakout', test_date)

print(f"\n📊 CLI Results:")
print(f"  • Results count: {len(cli_results)}")
print(f"  • Results type: {type(cli_results)}")

if cli_results:
    print(f"  • Sample result: {cli_results[0] if cli_results else 'None'}")
    
    # Convert to DataFrame for display
    cli_df = pd.DataFrame(cli_results)
    print(f"\n📋 CLI DataFrame shape: {cli_df.shape}")
    
    if not cli_df.empty:
        print("\n📊 CLI Results Preview:")
        print(cli_df.head().to_string(index=False))
else:
    print("  • No results from CLI execution")

# Show CLI command structure
print(f"\n💻 CLI Command Structure:")
print(f"  cd {project_root}")
print(f"  python -c \"from src.app.startup import get_scanner; ...\"")

print("\n✅ CLI integration demo complete!")

# 🎯 Key Features of Unified Integration

This updated dashboard provides:
- **Unified DuckDB Integration**: Uses the new unified database layer
- **CLI-Style Execution**: Can run scanners via command line
- **Connection Pooling**: Efficient database connection management
- **Intelligent Caching**: Result caching with configurable TTL
- **Error Handling**: Robust error handling and fallback mechanisms
- **Interactive Visualizations**: Dark mode plotly dashboards
- **Real-time Performance**: Live connection pool and cache statistics

All visualizations are interactive and can be zoomed, hovered, and filtered.